In [1]:
from pathlib import Path
import os

STUDENT = 'mmr497'
DATA_PATH = Path('/local/data/mmr497')
OUTLIERS_PATH = Path('./outliers2/')
INTERMEDIATE_PATH = Path(f'{DATA_PATH}/intermediate_datafiles/')
os.chdir(f'/home/{STUDENT}/')
from util.VisualizeDataset import VisualizeDataset
from Visualiser import Visualiser as Viz
from util.util import ignore_actual_time, read_parquet, write_parquet
from FeatureCreator import FeatureCreatorUpdated
from DataLearningLoader import DataLearningLoader
import pandas as pd
from Chapter7.FeatureSelection import FeatureSelectionClassification
from Chapter7.LearningAlgorithms import ClassificationAlgorithms
from AlteredAlgorithms import AlteredAlgorithmsClassifier
from Chapter7.Evaluation import ClassificationEvaluation
import numpy as np
from CustomPipeline import PreConfiguredPipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
# Testing parameters
feature_selection = True

In [3]:
EasyViz = Viz()
DataViz = VisualizeDataset('ML2.ipynb')

In [13]:
cleaned_dataset = read_parquet(INTERMEDIATE_PATH / 'ML4QS_imputed_results.parquet')

In [35]:
instance_mask = cleaned_dataset.id == 1
col = 'location_phone_Velocity'
print(cleaned_dataset.loc[instance_mask, col].isna().sum() / len(cleaned_dataset.loc[instance_mask, col]))
if cleaned_dataset.loc[instance_mask, col].isna().all():
    print('All values are NaN for instance {}.'.format(cleaned_dataset.loc[instance_mask, 'id'].values[0]))

1.0
All values are NaN for instance 1.


In [9]:
cleaned_dataset.dtypes

id                                              int64
acc_phone_X                                   float64
acc_phone_Y                                   float64
acc_phone_Z                                   float64
lin_acc_phone_X                               float64
lin_acc_phone_Y                               float64
lin_acc_phone_Z                               float64
gyr_phone_X                                   float64
gyr_phone_Y                                   float64
gyr_phone_Z                                   float64
location_phone_Latitude                       float64
location_phone_Longitude                      float64
location_phone_Height                         float64
location_phone_Velocity                       float64
location_phone_Direction                      float64
location_phone_Horizontal Accuracy            float64
location_phone_Vertical Accuracy              float64
mag_phone_X                                   float64
mag_phone_Y                 

In [6]:
test_df = read_parquet(INTERMEDIATE_PATH / 'non_fourier_features.parquet')
nan_counts = test_df.isna().sum()
nan_cols = nan_counts[nan_counts > 0]
print(nan_cols)

location_phone_Latitude                      45012
location_phone_Longitude                     45012
location_phone_Height                        45090
location_phone_Velocity                      45345
location_phone_Direction                     45555
location_phone_Horizontal Accuracy           45012
location_phone_Vertical Accuracy             45012
mag_phone_X                                   1631
mag_phone_Y                                   1631
mag_phone_Z                                   1631
proximity_phone_Distance                     52950
location_phone_Velocity_mean                 20205
location_phone_Velocity_median               20205
location_phone_Velocity_min                  20205
location_phone_Velocity_max                  20205
location_phone_Direction_mean                20465
location_phone_Direction_median              20465
location_phone_Direction_min                 20465
location_phone_Direction_max                 20465
location_phone_Horizontal Accur

In [7]:
# fcr = FeatureCreatorUpdated(INTERMEDIATE_PATH)
# DLL = DataLearningLoader(df_path=INTERMEDIATE_PATH, output_dir=INTERMEDIATE_PATH, verbose=False)

In [8]:
# feature_df = fcr.create_features(cleaned_dataset, overwrite=True)

Calculating magnitude for acc_phone...
timestamp
1970-01-01 00:00:00.000    0.595657
1970-01-01 00:00:00.250    0.142101
1970-01-01 00:00:00.500   -0.165194
1970-01-01 00:00:00.750   -0.393992
1970-01-01 00:00:01.000   -0.447120
Name: acc_phone_X, dtype: float64
timestamp
1970-01-01 00:00:00.000    3.355480
1970-01-01 00:00:00.250    3.311633
1970-01-01 00:00:00.500    3.236582
1970-01-01 00:00:00.750    3.161564
1970-01-01 00:00:01.000    3.255967
Name: acc_phone_Y, dtype: float64
timestamp
1970-01-01 00:00:00.000    9.099531
1970-01-01 00:00:00.250    9.272894
1970-01-01 00:00:00.500    9.294871
1970-01-01 00:00:00.750    9.303603
1970-01-01 00:00:01.000    9.296580
Name: acc_phone_Z, dtype: float64
Calculating magnitude for lin_acc_phone...
timestamp
1970-01-01 00:00:00.000    0.058131
1970-01-01 00:00:00.250    0.017935
1970-01-01 00:00:00.500    0.009399
1970-01-01 00:00:00.750   -0.027900
1970-01-01 00:00:01.000    0.031913
Name: lin_acc_phone_X, dtype: float64
timestamp
1970-01-


KeyboardInterrupt



In [9]:
# nan_counts = feature_df.isna().sum()
# nan_cols = nan_counts[nan_counts > 0]
# print(nan_cols)

In [10]:
# # Stupid fix! Only temporary! MUST REMOVE
# def remove_nan_cols(df):
#     df_fs = df.copy()
#     col_before = df_fs.columns
#     nan_counts = df_fs.isna().sum()
#     nan_cols = nan_counts[nan_counts > 0]
#     df_fs = df_fs.drop(nan_cols.index, axis=1)
#     col_after = df_fs.columns
#     print('Removed columns: ', [col for col in col_before if col not in col_after])
#     return df_fs
#
# filtered_feature_df = remove_nan_cols(feature_df)

In [11]:
# X_train, X_test, y_train, y_test = DLL.prepare_data(filtered_feature_df)

In [2]:
intermediate_df = read_parquet(INTERMEDIATE_PATH / 'ML4QS_combined_results_example.parquet')

In [5]:
intermediate_df.dtypes

id                                      int64
acc_phone_X                           float64
acc_phone_Y                           float64
acc_phone_Z                           float64
lin_acc_phone_X                       float64
lin_acc_phone_Y                       float64
lin_acc_phone_Z                       float64
gyr_phone_X                           float64
gyr_phone_Y                           float64
gyr_phone_Z                           float64
location_phone_Latitude               float64
location_phone_Longitude              float64
location_phone_Height                 float64
location_phone_Velocity               float64
location_phone_Direction              float64
location_phone_Horizontal Accuracy    float64
location_phone_Vertical Accuracy      float64
mag_phone_X                           float64
mag_phone_Y                           float64
mag_phone_Z                           float64
proximity_phone_Distance              float64
labeltram                         

In [ ]:
pipe = PreConfiguredPipeline(intermediate_path=INTERMEDIATE_PATH)
X_train, X_test, y_train, y_test = pipe.fit_transform(intermediate_df, verbose=True, overwrite=True)

[INFO] Removing bad sensors...
[INFO] Splitting data into train and test sets...
[INFO] Starting preprocessing pipeline...
[INFO] Preprocessing X_train...
[INFO] Checking for outliers...


bike: Only 2 sessions - 1 train, 1 test


[INFO] Ensuring correct data types...
[INFO] Imputing missing values...


<class 'pandas.core.frame.DataFrame'>
No observed values for location_phone_Direction for vehicle type l in instance timestamp
1970-01-01 00:00:00.000    False
1970-01-01 00:00:00.250    False
1970-01-01 00:00:00.500    False
1970-01-01 00:00:00.750    False
1970-01-01 00:00:01.000    False
                           ...  
1970-01-01 03:40:59.500    False
1970-01-01 03:40:59.750    False
1970-01-01 03:41:00.000    False
1970-01-01 03:41:00.250    False
1970-01-01 03:41:00.500    False
Name: id, Length: 32564, dtype: bool.
No observed values for location_phone_Direction for vehicle type l in instance timestamp
1970-01-01 00:00:00.000    False
1970-01-01 00:00:00.250    False
1970-01-01 00:00:00.500    False
1970-01-01 00:00:00.750    False
1970-01-01 00:00:01.000    False
                           ...  
1970-01-01 03:40:59.500    False
1970-01-01 03:40:59.750    False
1970-01-01 03:41:00.000    False
1970-01-01 03:41:00.250    False
1970-01-01 03:41:00.500    False
Name: id, Length: 32

[INFO] Creating features...


In [ ]:
X_train.columns.startswith("label")

In [28]:
# Optional for testing

def remove_problem_cols(df):
    # This is for testing purposes, stupid fix!
    df_fs = df.copy()
    col_before = df_fs.columns
    df_fs = df_fs.select_dtypes(include=[np.number], exclude=['timedelta64[ns]'])
    col_after = df_fs.columns
    print('Removed columns: ', [col for col in col_before if col not in col_after])
    return df_fs

if feature_selection:
    fs = FeatureSelectionClassification()

    N_FORWARD_SELECTION = 50

    X_train_fs = remove_problem_cols(X_train)
    X_test_fs = remove_problem_cols(X_test)

    features, ordered_features, ordered_scores = fs.forward_selection(N_FORWARD_SELECTION,
                                                                      X_train_fs,
                                                                      X_test_fs,
                                                                      y_train,
                                                                      y_test,
                                                                      gridsearch=True)

    DataViz.plot_xy(x=[range(1, N_FORWARD_SELECTION+1)], y=[ordered_scores],
                xlabel='number of features', ylabel='accuracy')

Removed columns:  ['time_diff', 'shifted_time']
Removed columns:  ['time_diff', 'shifted_time']
Added feature0


KeyboardInterrupt: 

In [29]:
# Setup model
CA = ClassificationAlgorithms()
alt_CA = AlteredAlgorithmsClassifier()

X_train_fs = remove_problem_cols(X_train)
X_test_fs = remove_problem_cols(X_test)

# First MLP

mlp_train_y, mlp_test_y, mlp_train_prob_y, mlp_test_prob_y = alt_CA.feedforward_neural_network(
            X_train_fs, y_train, X_test_fs, gridsearch=True
)

# Train RF

rf_train_y, rf_test_y, rf_train_prob_y, rf_test_prob_y = alt_CA.random_forest(
    X_train_fs, y_train, X_test_fs, gridsearch=True
)


Removed columns:  ['time_diff', 'shifted_time']
Removed columns:  ['time_diff', 'shifted_time']



KeyboardInterrupt



In [ ]:
# Evaluate models


CEVAL = ClassificationEvaluation()

CEVAL.f1(y_train, mlp_train_y)